In [8]:
# Limpeza e processamento dos Dados
import pandas as pd

# Carregar e fazer leitura dos dados dentro do projeto
caminho_do_arquivo = "../desafio-ds-imdb-diel/desafio_indicium_imdb.csv"
data = pd.read_csv(caminho_do_arquivo)
